In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install smote_variants

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.2/416.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.7/200.7 MB 6.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 47.0 MB/s eta 0:00:00:00:0100:01
  Created wheel for minisom: filename=MiniSom-2.3.1-py3-none-any.whl size=10589 sha256=b2c58a2e078cd031fe4f280b2be7029249624c40400b6acf72d0851c39df53a3
  Stored in directory: /root/.cache/pip/wheels/c7/92/d2/33bbda5f86fd8830510b16aa98c8dd420129b5cb24248fd6db
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7436 sha256=1f27b0b91d7c811dae7e0beab0d5c1baca422b47a86c7abd8c71ba496f5f8b97
  Stored in directory: /root/.cache/pip/wheels/59/bd/aa/41ad56fbb723d36e457b5be74feb992cdac2855a6999ba34a1
Successfully built minisom statistics
Note: you may n

In [4]:
import smote_variants as sv

In [6]:
oversampler= sv.Borderline_SMOTE1()

In [2]:
normal_files=os.listdir('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL')

print(normal_files[0:5])
print(normal_files[-5:])

['NORMAL2-IM-0771-0001.jpeg', 'NORMAL2-IM-1294-0001-0002.jpeg', 'IM-0675-0001.jpeg', 'NORMAL2-IM-1169-0001.jpeg', 'IM-0421-0001.jpeg']
['NORMAL2-IM-1160-0001.jpeg', 'NORMAL2-IM-0389-0001.jpeg', 'IM-0162-0001.jpeg', 'NORMAL2-IM-1247-0001.jpeg', 'IM-0219-0001.jpeg']


In [ ]:
Pneumonia_files=os.listdir('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA')

print(Pneumonia_files[0:5])
print(Pneumonia_files[-5:])

In [ ]:
Normal_label=[0]*1341
Pneumonia_label=[1]*3875

In [ ]:
print('With Normal labels: ', Normal_label[0:5])
print('Without Pneumonia labels: ', Pneumonia_label[0:5])

In [ ]:
labels=Normal_label + Pneumonia_label

print('Labels are: ',len(labels))

print(labels[0:5])
print(labels[-5:])

In [ ]:
from PIL import Image

normal_path=('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/')
data=[]

for img_file in normal_files:
    image=Image.open(normal_path +img_file)
    image=image.resize((128,128))
    image=image.convert('RGB')
    image=np.array(image)
    data.append(image)
    
Pneumonia_path=('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/')

for img_file in Pneumonia_files:
    image=Image.open(Pneumonia_path + img_file )
    image=image.resize((128,128))
    image=image.convert('RGB')
    image=np.array(image)
    data.append(image)

In [ ]:
import numpy as np

from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from ..base import OverSampling, NearestNeighborsWithMetricTensor, coalesce
from .._logger import logger
from ._smote import SMOTE
import math
_logger = logger

__all__ = ["SMOTE_Cosine_Eng"]


class SMOTE_Cosine_Eng(OverSampling):
    categories = [
        OverSampling.cat_noise_removal,  # applies noise removal
        OverSampling.cat_uses_classifier,  # uses some advanced classifier
        OverSampling.cat_sample_ordinary,  # sampling is done in the SMOTE scheme
        OverSampling.cat_extensive,  # adds minority samples only
        OverSampling.cat_metric_learning,
    ]  # metric learning is applicable (uses nearest neighbors)
    sample_good = 1
    sample_lonely = 0
    sample_bad = -1
    def __init__(
        self, proportion=1.0,
            n_neighbors=5, *,
            n_jobs=1,
            random_state=None,
            nn_params=None,
            n_iters=100,
            max_depth=30,
            **_kwargs
    ):
        
        super().__init__(random_state=random_state)
        self.check_greater_or_equal(proportion, "proportion", 0)
        self.check_greater_or_equal(n_neighbors, "n_neighbors", 1)
        self.check_n_jobs(n_jobs, "n_jobs")

        self.proportion = proportion
        self.n_neighbors = n_neighbors
        self.max_depth = max_depth
        self.nn_params = coalesce(nn_params, {})
        self.n_iters = n_iters
        self.n_jobs = n_jobs

    @classmethod
    def parameter_combinations(cls, raw=False):
        """
        Generates reasonable parameter combinations.

        Returns:
            list(dict): a list of meaningful parameter combinations
        """
        parameter_combinations = {
            "proportion": [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0],
            "n_neighbors": [3, 5, 7],
        }
        return cls.generate_parameter_combinations(parameter_combinations, raw)

    def epsilon_neighborhood_graph(self, X, epsilon=0.5):
        """
        Build an epsilon-neighborhood graph based on cosine similarity.

        Args:
            X (np.ndarray): Data points.
            epsilon (float): Threshold for considering neighbors.

        Returns:
            np.ndarray: Binary matrix representing the epsilon-neighborhood graph.
        """
        n_samples = X.shape[0]
        graph = np.zeros((n_samples, n_samples), dtype=bool)

        for i in range(n_samples):
            for j in range(i + 1, n_samples):
                cosine_similarity = np.dot(X[i], X[j]) / (np.linalg.norm(X[i]) * np.linalg.norm(X[j]))
                if cosine_similarity >= epsilon:
                    graph[i, j] = graph[j, i] = True

        return graph
    def first_majority_index(self, nearest_ys, no_majority_result=-1):
        """
        Finds the index of the first majority element in the nearest_ys per row.

        Args:
            nearest_ys (np.ndarray): label of the nearest neighbours
            no_majority_result (int): result if there is no majority item in a row
        Returns:
            np.array: the i-th element of the array is the index of the first majority label
                        in the i-th row of the nearest_ys, or no_majority_result if the row
                        contains no majority labels.
        """
        mask = nearest_ys == self.maj_label
        return np.where(mask.any(axis=1), mask.argmax(axis=1), no_majority_result)
    def sampling_formula(self, u, v, x):  # pylint: disable=invalid-name
        """
        The sampling formula.

        Args:
            u (np.array): the u vector
            v (np.array): the v vector
            x (np.array): the x vector

        Returns:
            np.array: the generated samples
        """
        uu = u + self.random_state.random_sample(u.shape) * 0.3 * (
            u - v
        )  # pylint: disable=invalid-name

        return x + self.random_state.random_sample(x.shape) * 0.5 * (uu - x)

    def generate_samples(
        self, *, X, y, X_maj, X_min, n_to_sample, nn_maj_ind, composite_ind
    ):
        """
        Generate samples

        Args:
            X (np.array): all training vectors
            y (np.array): the target labels
            X_maj (np.array): majority vectors
            X_min (np.array): minority vectors
            n_to_sample (int): number of samples to generate
            nn_maj_ind (np.array): majority neighborhood structure
            composite_ind (np.array): composite neighborhood structure

        Returns:
            np.array: the generated samples
        """
        minority_indices = np.where(y == self.min_label)[0]

        base_ind = self.random_state.choice(
            np.arange(len(minority_indices)), n_to_sample
        )

        neigh_ind = self.random_state.choice(
            np.arange(0, nn_maj_ind.shape[1]), n_to_sample
        )

        min_neigh_ind = self.random_state.choice(
            np.arange(1, composite_ind.shape[1]), n_to_sample
        )

        return self.sampling_formula(
            u=X[minority_indices[base_ind]],
            v=X_maj[nn_maj_ind[base_ind, neigh_ind]],
            x=X_min[composite_ind[base_ind, min_neigh_ind]],
        )
    @staticmethod
    def noise_threshold(w, th):
        """
        Creating the noise mask

        Args:
            w (np.array): the weights
            th (float): the threshold

        Returns:
            np.array: the mask
        """
        noise = np.repeat(False, len(w))
        noise[w > th] = True
        return noise

    def noise_detection(self, X, y):
        """
        Implementation of Algorithm 3.

        Args:
            X (np.ndarray): training set
            y (np.array): target labels

        Returns:
            (np.array, np.array): Mask of noise for the minority and the majority
            samples, respectively.
        """

        w = self.boosted_weights(X, y)

        w_min = w[y == self.min_label]
        w_maj = w[y == self.maj_label]

        n_samples = len(y)

        # noise thresholds for majority and minority classes
        th_min = (2 * len(w_maj)) / (n_samples**2)
        th_maj = (2 * len(w_min)) / (n_samples**2)

        noise_mask_min = SMOTE_Cosine_Eng.noise_threshold(w_min, th_min)
        noise_mask_maj = SMOTE_Cosine_Eng.noise_threshold(w_maj, th_maj)

        return noise_mask_min, noise_mask_maj
    def sampling_algorithm(
            self, X, y
    ):  # pylint: disable=too-many-locals,too-many-statements
        """
        Does the sample generation according to the class parameters.

        TODO: break into smaller functions

        Args:
            X (np.ndarray): training set
            y (np.array): target labels
        Returns:
            (np.ndarray, np.array): the extended training set and target labels
        """

        # determine the number of samples to generate
        n_to_sample = self.det_n_to_sample(self.proportion)

        if n_to_sample == 0:
            return self.return_copies(X, y, "Sampling is not needed.")

        # use logging in the below format
        _logger.info("%s: sampling", self.__class__.__name__)

        # Step 1: MinMaxScaler - Rescaling is ignored at the suggestion of the
        # author of the algorithm.

        # Step2: separating the classes
        X_min = X[y == self.min_label]
        X_maj = X[y == self.maj_label]

        # Step 3-4: creating masks of noise samples
        noise_mask_min, noise_mask_maj = self.noise_detection(X, y)

        # Start: Algorithm 4
        n_min = len(X_min)
        n_maj = len(X_maj)

        # paper: floor, R implementation: ceil
        k_max = math.floor(n_maj / n_min)

        # non-noise samples and labels
        X_min_not_noise = X_min[~noise_mask_min]  # pylint: disable=invalid-name
        X_maj_not_noise = X_maj[~noise_mask_maj]  # pylint: disable=invalid-name
        X_not_noise = np.concatenate(  # pylint: disable=invalid-name
            [X_min_not_noise, X_maj_not_noise]
        )
        y_not_noise = np.concatenate(
            [
                np.repeat(self.min_label, X_min_not_noise.shape[0]),
                np.repeat(self.maj_label, X_maj_not_noise.shape[0]),
            ]
        )

        # fitting the model
        # paper: X^good_pos makes no sense, R code: x_notnoise
        nn_params = {**self.nn_params}
        nn_params["metric_tensor"] = self.metric_tensor_from_nn_params(nn_params, X, y)

        # all data points are noise
        if len(X_not_noise) == 0:
            smote = SMOTE(proportion=self.proportion, nn_params=self.nn_params)
            return smote.sample(X, y)

        n_neighbors = min([len(X_not_noise), k_max + 1])
        nnmt = NearestNeighborsWithMetricTensor(
            n_neighbors=n_neighbors, n_jobs=self.n_jobs, **nn_params
        )

        nnmt.fit(X_not_noise)
        indices = nnmt.kneighbors(X_min, n_neighbors=n_neighbors, return_distance=False)

        # Build epsilon-neighborhood graph based on cosine similarity
        epsilon_graph = self.epsilon_neighborhood_graph(X_min, epsilon=0.5)
        composite_ind = epsilon_graph.argsort(axis=1)
        # number of the positive neighbors until the first negative one (per row)
        k_arr = self.first_majority_index(
            y_not_noise[indices], no_majority_result=k_max + 1
        )

        # decrease one if the sample is not a noise
        k_arr[~noise_mask_min] -= 1
        # in case there is a majority sample at 0 distance
        k_arr[k_arr == -1] = 0
        # all neighbours are positive
        k_max = min(indices.shape[1], k_max)
        k_arr[k_arr > k_max] = k_max

        # setting the labels of the samples of X_pos
        fl_arr = np.empty(n_min, int)
        fl_arr[k_arr > 0] = SMOTE_Cosine_Eng.sample_good
        fl_arr[(k_arr == 0) & noise_mask_min] = SMOTE_Cosine_Eng.sample_bad
        fl_arr[(k_arr == 0) & (~noise_mask_min)] = SMOTE_Cosine_Eng.sample_lonely
        # End: Alg 4

        # Step 5: n_to_sample - done
        # Step 6: computing n_to_sample per samples
        n_min = len(X_min)

        # number of good and lonely samples
        n_good_min = np.sum(fl_arr == SMOTE_Cosine_Eng.sample_good)
        n_lonely_min = np.sum(fl_arr == SMOTE_Cosine_Eng.sample_lonely)

        # prevent errors (missing from the original) no good or lonely
        # samples => give chance to the others
        if n_good_min + n_lonely_min == 0:
            samples = self.generate_samples(
                X=X,
                y=y,
                X_maj=X_maj,
                X_min=X_min,
                n_to_sample=n_to_sample,
                nn_maj_ind=nnmt,
                composite_ind=composite_ind,
            )
            return samples
            #smote = SMOTE(proportion=self.proportion, nn_params=self.nn_params)
            #return smote.sample(X, y)

        n_to_sample_per_sample = math.ceil(n_to_sample / (n_good_min + n_lonely_min))
        C = np.zeros(n_min, int)  # pylint: disable=invalid-name
        C[fl_arr == SMOTE_Cosine_Eng.sample_good] = n_to_sample_per_sample
        C[fl_arr == SMOTE_Cosine_Eng.sample_lonely] = n_to_sample_per_sample

        # Step 8: correcting the number of samples to be generated to achieve the desired balance
        # paper: ceil => n_to_sample-p.sum(C) zero or negative
        # we probably have too many samples because of the ceil
        diff = np.sum(C) - n_to_sample
        if diff > 0:
            good_and_lonely_ind = np.where(
                (fl_arr == SMOTE_Cosine_Eng.sample_good) | (fl_arr == SMOTE_Cosine_Eng.sample_lonely)
            )[0]
            selected_ind = self.random_state.choice(
                good_and_lonely_ind, diff, replace=len(good_and_lonely_ind) < diff
            )
            C[selected_ind] -= 1

        # Step 9: sample generation
        synt_sample_list = []
        for i in range(0, n_min):
            if fl_arr[i] == SMOTE_Cosine_Eng.sample_lonely:
                for j in range(C[i]):
                    synt_sample_list.append(X_min[i].copy())
            elif fl_arr[i] == SMOTE_Cosine_Eng.sample_good and C[i] > 0:
                nn = (
                    indices[i, 0 : k_arr[i]]
                    if noise_mask_min[i]
                    else indices[i, 1 : k_arr[i] + 1]
                )

                if len(nn) > 0:
                    k_ids = self.random_state.choice(nn, C[i])
                    for j in k_ids:
                        synt_sample_list.append(
                            self.sample_between_points(X_min[i], X_not_noise[j])
                        )

        # Step 11-12: merging the original samples and the synthetic ones
        X_synt_samples = np.array(synt_sample_list)  # pylint: disable=invalid-name
        X_samples = np.vstack([X, X_synt_samples])  # pylint: disable=invalid-name

        return (
            X_samples,
            np.hstack([y, np.hstack([self.min_label] * len(synt_sample_list))]),
        )
    def boosted_weights(self, X, y):
        """
        Implementation of Algorithm 2.

        Args:
            X (np.ndarray): training set
            y (np.array): target labels

        Returns:
            (np.array): weight of the samples
        """
        n_samples = X.shape[0]
        w = np.repeat(1.0 / n_samples, n_samples)

        # based on boosted_weights.R
        clf = DecisionTreeClassifier(
            random_state=self._random_state_init,
            max_depth=self.max_depth,
            min_samples_split=3,
        )

        for _ in range(0, self.n_iters):
            clf.fit(X, y, sample_weight=w)
            predicted_labels = clf.predict(X)

            w_error = sum(w[predicted_labels != y])

            # no zero check in the original code (samples are not_noise)
            if w_error == 0:
                return np.zeros(n_samples, float)

            alpha = 0.5 * np.log((1 - w_error) / w_error)
            w[predicted_labels != y] = w[predicted_labels != y] * np.exp(alpha)

            w = w / sum(w)

        return w
    def get_params(self, deep=False):
        """
        Returns:
            dict: the parameters of the current sampling object
        """
        return {
            "proportion": self.proportion,
            "n_neighbors": self.n_neighbors,
            "n_jobs": self.n_jobs,
            **OverSampling.get_params(self),
        }


In [ ]:
import numpy as np
from skimage import io
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from your_module import SMOTE_Cosine_Eng  # Import your SMOTE-Cosine module

# Load image data (as an example)
data = fetch_openml(name='mnist_784', version=1)
X, y = data['data'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data to images (this is a simplified example, actual reshaping may vary)
X_train_images = X_train.reshape(-1, 28, 28)
X_test_images = X_test.reshape(-1, 28, 28)

# Create an instance of SMOTE_Cosine_Eng
smote_cosine = SMOTE_Cosine_Eng()

# Apply SMOTE-Cosine oversampling to image data
X_train_oversampled, y_train_oversampled = smote_cosine.fit_resample(X_train_images, y_train)
